<a href="https://colab.research.google.com/github/JGS2020-012/text_analysis/blob/master/calcNegaPosiScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 必要なライブラリをインポート
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
import pathlib
import pandas as pd 

# added kamegai
import requests 
import io

In [0]:
def before_filter(before_st):
    """
    文書を渡すとjanomeで処理してリストで返す
    （山崎さんのプログラムを拝借）
    """  
    #Unicode正規化・半角記号除去・全角記号除去・＜＞タグ除去・改行除去
    char_filters = [UnicodeNormalizeCharFilter('NFKC'),
                    RegexReplaceCharFilter('[!/:%#\$&\?\(\)~\.=\+\-…]+', ''),
                    RegexReplaceCharFilter('[『！”＃＄％＆’（）＝～｜‘｛＋＊｝＜＞？＿－＾￥＠「；：」、。・]', ''),
                    RegexReplaceCharFilter('<.*?>', ''), 
                    RegexReplaceCharFilter('[\n|\r|\t]', '')
                   ]
    #名詞だけ取得・アルファベット小文字化・基本形
    token_filters = [
                     #POSKeepFilter(['名詞']),
                     LowerCaseFilter(),
                     ExtractAttributeFilter('base_form')]
    #設定からフィルターを作る
    a = Analyzer(char_filters=char_filters, token_filters=token_filters)
    #文章をanalyzerに掛け処理する
    after_st = a.analyze(before_st)
    
    #ストップワードを除去する
    stop_words = []

    # modified kamegai
    #path = 'Japanese.txt'
    #with open(path) as f:
    #    stop_words = f.read().split('\n')
    stop_word_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
    stop_word_res = requests.get(stop_word_url).content
    stop_word_io = io.StringIO(stop_word_res.decode('utf-8'))
    stop_words = stop_word_io.read().split('\n')

    after_st = [x for x in after_st if x not in stop_words]

    return after_st

In [0]:
def load_dict(url_file):
    """
    極性表現辞書をロードする。
    """  
    res = requests.get(url_file).content
    lists = []
    reader = csv.reader(io.StringIO(res.decode('utf-8')), delimiter='\t')
    for row in reader:
        #print(reader.line_num,row[0])
        lists.append(row)
    return lists

In [0]:
# 極性表現辞書を基にスコアリングできる状態にする。

def get_sentiment_pn_dict():
    """
    日本語評価極性辞書（用言編）
           1                  2　
    (例) ネガ（評価）	恩 着せ が まし い
    (方針)・1列目のネガ（評価）は-1、ポジ（評価）は1へ変換する。
    　    ・2列目は、文字を連結して、形態素解析を行う。
    """
    dict_content = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn')

def get_sentiment_trim_dict():
    """
    日本語評価極性辞書（名詞編）
       　   1     2      3　
    (例) ２か月   e	  〜である・になる（状態）客観
    (方針)・2列目のnは-1、pは1へ変換する。（p:ポジティブ、e:ニュートラル、n:ネガティブ）
          ・3列目に・を含む場合は分割し、～を1列目で置換する。
    """   
    dict_content = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim')



In [0]:
# ネガポジ表現数を基にスコアリング
def calc_score(input_text):
  preprocess()

In [27]:
# テストプログラム
twiite_sample = "LINEpay使えない店でもpayカードならクレカ扱ってるとこはどこでも使えるし便利だよ〜考えてみて！"
before_filter(twiite_sample)


['linepay',
 '使える',
 'ない',
 '店',
 'で',
 'も',
 'pay',
 'カード',
 'だ',
 'クレカ',
 '扱う',
 'てる',
 'とこ',
 'は',
 'どこ',
 'でも',
 '使える',
 'し',
 '便利',
 'だ',
 'よ',
 '〜',
 '考える',
 'て',
 'みる',
 'て']

     |████████████████████████████████| 21.5MB 1.3MB/s 
